In [2]:
pip install netcdf4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.3 MB/s eta 0:00:00


In [3]:
%pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 38.8 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [11]:
import requests
import pandas as pd
from netCDF4 import Dataset, num2date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import os
from shapely.geometry import Polygon, Point
import geopandas as gpd
import folium
import plotly.graph_objects as go
from shapely.geometry import Point, box

In [6]:
def prepararDF(df, codigo):
  # Muda o tipo da coluna 'Data' para datetime (string --> datetime)
  df['Data'] = df['DataHora'][:10]
  df['Data'] = pd.to_datetime(df['Data'], format='%Y/%m/%d')

  # A coluna 'Data' e 'NivelConsistencia' são usadas como índice
  df.set_index(['Data','NivelConsistencia'], inplace=True)

  # Exclui colunas desnecessárias do dataframe
  listColumnsDrop = []
  for i in range(77):
    if i > 44:
      listColumnsDrop += [i]
  listColumnsDrop += [0,1,2,3,4,5,6,7,8,9,10,11,12,13]
  df = df.drop(df.columns[listColumnsDrop], axis=1)
  
  # Avalia se existe uma mesma data com diferentes níveis de consistência
  # Classificar o dataframe por data
  df.sort_values("Data", inplace=True)

  # Reseta os índices para passar "Data" como coluna e utilizar a mesma para apagar as linhas duplicadas
  df = df.reset_index()

  # apagas as linhas duplicadas, dando preferência ao dado consistido ('NivelConsistencia' = 2)
  df = df.drop_duplicates(subset='Data', keep='last', inplace=False)

  # A coluna 'Data' e 'NivelConsistencia' são usadas como índice
  df.set_index(['Data','NivelConsistencia'], inplace=True)

  # transforma as colunas de chuva em linhas com as respectivas datas
  # Este processo transforma todas as colunas em linhas, gerando um problema para os meses que possuem menos de 31 dias
  # Dessa forma, o dia 1 de um mês armazenava a chuva do dia 1 e a chuva do dia 31 (linha duplicada com valor NaN)
  # Correção explicada durante o processo:

  # usa a função .melt() para transformar todas as colunas em linhas. São mantidas as colunas ['Data', 'NivelConsistencia']
  # São criadas duas novas colunas: 'Dia' que recebe o nome da coluna que virou linha e 'Chuva' que recebe o valor que era armazenado na coluna
  df = df.reset_index().melt(id_vars=['Data', 'NivelConsistencia'], var_name='Dia', value_name='Chuva')

  # Fiz uma cópia da minha data em formato de texto
  df['Data1']=df['Data'].astype(str)

  # Apaguei os últimos 2 caracteres, assim ficou somente o texto com %Y-%m- ('1989-11-')
  df['Data1'] = df['Data1'].str[:-2]

  # Fiz uma cópia da coluna 'Dia' (exe: 'Chuva03'), a 'função .str.extract('(\d+)', expand=False)' deixa apenas os números
  # (então a coluna 'Dia1' armazena a string '03') ('Chuva03' --> '03')  
  df['Dia1'] = df['Dia'].str.extract('(\d+)', expand=False)
  
  # praticamente a mesma coisa da linha anterior, mudando apenas que substitui a própria coluna e o formato é int ('Chuva03' --> 3)
  df['Dia'] = df['Dia'].str.extract('(\d+)', expand=False).astype(int)

  # Como foram criadas várias linhas, tem vários índices repetidos da data, esta função soma os dias correspondente
  # aos da chuva guardado na coluna 'Dia' na data do índice (aqui é gerado o erro dos 31 dias)
  df['Data'] = df.apply(lambda x: x['Data'] + pd.DateOffset(days=x['Dia']-1), axis=1)

  # Aqui é feita uma concatenação do tipo '1989-11-'+'03' formando uma data.
  # Mas aqui também forma datas como '1989-02-31'
  df['Data1'] = df['Data1'] + df['Dia1']

  # A maioria das datas das colunas 'Data' e 'Data1' serão iguais, mas datas como '1989-02-31'
  # vão gerar valor NaT pois é um erro (exatamente nas linhas duplicadas)
  df['Data1'] = pd.to_datetime(df['Data1'], format='%Y-%m-%d', errors='coerce')

  # Aqui é apagada todas as linhas em que o erro NaT aparece na coluna 'Data1' (removendo os valores duplicados)
  df.dropna(inplace=True, subset=['Data1'])

  # reorganiza as colunas
  df = df[['Data', 'NivelConsistencia', 'Chuva']]

  # Classificar o dataframe por data
  df.sort_values("Data", inplace=True)

  # A coluna 'Data' e 'NivelConsistencia' são usadas como índice novamente
  #df.set_index(['Data'], inplace=True)
  df['Codigo'] = codigo
  return df

In [7]:
# API para solicitar dados de estações sem telemetria
estacoes = ['936045']
frames = []
for estacao in estacoes:
  url = 'http://telemetriaws1.ana.gov.br/ServiceANA.asmx/HidroSerieHistorica'
  params = {
      'codEstacao': estacao,
      'dataInicio': '',
      'dataFim': '',
      'tipoDados': '2',
      'nivelConsistencia': '',
  }
  try:
    response = requests.get(url, params=params)
    xml = response.text
    df1 = pd.read_xml(xml,
                      xpath="//SerieHistorica",
                      namespaces={"xs": "http://www.w3.org/2001/XMLSchema"})
  except ValueError:
    pass
  else:
    response = requests.get(url, params=params)
    xml = response.text
    df1 = pd.read_xml(xml,
                      xpath="//SerieHistorica",
                      namespaces={"xs": "http://www.w3.org/2001/XMLSchema"})
    df1 = prepararDF(df1, estacao)
    df1 = df1.dropna(subset=['Chuva'])
    #df1.reset_index()
    frames += [df1]
    df_936045 = frames[0]
print(f'Foram carregadas {len(frames)} estações')

Foram carregadas 1 estações


<ipython-input-6-d8c20322755e>:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values("Data", inplace=True)
<ipython-input-6-d8c20322755e>:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Codigo'] = codigo


In [ ]:
df_936045_dez = df_936045.set_index("Data").loc['2000-12-01':'2000-12-31']
#periodo = periodo.reset_index("Data")
df_936045_dez

Missão GPM

In [9]:
# Diretório dos arquivos .nc4
diretorio = '/content/drive/MyDrive/ICD/Colab/Alexandre/GPM/'

# Coordenadas desejadas (X, Y)
lat = -9.1667
lon = -36.2167
	
# Criação do dataframe
df = pd.DataFrame(columns=['Date', 'Value'])

# Percorre os arquivos .nc4 no diretório
for arquivo in os.listdir(diretorio):
    if arquivo.endswith('.nc4'):
        # Caminho completo do arquivo
        caminho_arquivo = os.path.join(diretorio, arquivo)
        
        # Abre o arquivo NetCDF e carrega os dados
        ds = xr.open_dataset(caminho_arquivo)
        f1 = Dataset(caminho_arquivo)
        time = f1.variables["time"]
        ## Data for only one day
        dates = num2date(time[:], time.units)
        data = dates[0].strftime("%Y-%m-%d")
        # Obtém as datas dos dados
        #dates = num2date(ds.time[:], ds.time.units)
        
        # Converte a primeira data para o formato "YYYY-MM-DD"
        #data = dates[0].strftime('%Y-%m-%d')
        
        # Seleciona o valor de precipitação na coordenada (X, Y)
        valor = float(ds['HQprecipitation'].sel(lat=lat, lon=lon, method='nearest').values)
        
        # Adiciona a data e o valor ao dataframe
        df = df.append({'Date': data, 'Value': valor}, ignore_index=True)

df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')
# Imprime o dataframe resultante
#print(df)
df_nc4 = df

<ipython-input-9-df11c0f116f8>:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date': data, 'Value': valor}, ignore_index=True)
<ipython-input-9-df11c0f116f8>:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date': data, 'Value': valor}, ignore_index=True)
<ipython-input-9-df11c0f116f8>:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date': data, 'Value': valor}, ignore_index=True)
<ipython-input-9-df11c0f116f8>:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date': data, 'Value': valor}, ignore_index=True)
<ipython-input-9-df11c0f116f8>:34: FutureWarning: The fr

In [15]:
# Coordenadas desejadas (X, Y)
lat = -9.35
lon = -35.84

# Calcula a metade da resolução (15 km)
half_resolution = 0.15

# Calcula as coordenadas para formar o quadrado
lon_min = lon - half_resolution
lon_max = lon + half_resolution
lat_min = lat - half_resolution
lat_max = lat + half_resolution

# Cria um quadrado com base nas coordenadas
square = box(lon_min, lat_min, lon_max, lat_max)

# Cria um ponto com base nas coordenadas desejadas
point = Point(lon, lat)

# Cria um mapa folium centrado no centro do quadrado
m = folium.Map(location=[lat, lon], zoom_start=10)

# Adiciona o quadrado ao mapa folium
folium.GeoJson(square, name="Quadrado", tooltip="Pixel de 30x30km. Dados de 2000-12-01 até 2000-12-31").add_to(m)

# Adiciona o ponto ao mapa folium
folium.Marker(location=[lat, lon], tooltip="Estação 936045. Sem telemetria").add_to(m)

# Adiciona controle de camadas
folium.LayerControl().add_to(m)

# Exibe o mapa folium
m

Como pode ser visto no mapa, o pixel é grande e só tenho 1 mês de dados disponível.